In [3]:
#!/usr/bin/env python3
from collections import defaultdict  # to concatenate dictionaries
import matplotlib.ticker as plticker

import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm
import uproot3, uproot
import sys

np.set_printoptions(precision=3, suppress=True)

# TODO: use uniform row major OR column major format, not both ffs

def getIPnew(filename):
    fileTree = uproot.open(filename)['pndsim']

    dictIP = fileTree.arrays(
        [
            "LMDTrackQ.fTrkRecStatus",
            "LMDTrackQ.fXrec",
            "LMDTrackQ.fYrec",
            "LMDTrackQ.fZrec",
        ],
        library="np",
    )

    recStat = np.concatenate(dictIP["LMDTrackQ.fTrkRecStatus"]).ravel()
    recX = np.concatenate(dictIP["LMDTrackQ.fXrec"]).ravel()
    recY = np.concatenate(dictIP["LMDTrackQ.fYrec"]).ravel()
    recZ = np.concatenate(dictIP["LMDTrackQ.fZrec"]).ravel()

    mask = (recStat == 0)
    recXmask = recX[mask]
    recYmask = recY[mask]
    recZmask = recZ[mask]

    outarray = np.array([recXmask, recYmask, recZmask]).T

    return outarray

def getIPfromTrksQA(filename, cut=2.0, sensor=-1, module=-1, plane=-1, half=-1, maxTracks=0):

    resArray = np.zeros((6)).T

    try:
        # open the root trees in a TChain-like manner
        print(f'reading files {filename}...')
        for array in uproot3.iterate(filename, 'pndsim',
                                    ['LMDTrackQ.fTrkRecStatus', 'LMDTrackQ.fHalf', 'LMDTrackQ.fModule', 'LMDTrackQ.fXrec', 'LMDTrackQ.fYrec', 'LMDTrackQ.fZrec']):
            resArray = np.vstack((resArray, cleanArray(array)))

    except Exception as e:
        print('error occured:')
        print(e)
        sys.exit(0)

    return resArray


# removes all tracks with TrkRecStatus != 0
def cleanArray(arrayDict):

    recStatus = arrayDict[b'LMDTrackQ.fTrkRecStatus'].flatten()
    half = arrayDict[b'LMDTrackQ.fHalf'].flatten()
    module = arrayDict[b'LMDTrackQ.fModule'].flatten()
    recX = arrayDict[b'LMDTrackQ.fXrec'].flatten()
    recY = arrayDict[b'LMDTrackQ.fYrec'].flatten()
    recZ = arrayDict[b'LMDTrackQ.fZrec'].flatten()

    retArray = np.array([recStatus, half, module, recX, recY, recZ]).T
    recStatusMask = (recStatus == 0)

    return retArray[recStatusMask]


def extractIP(cleanArray, module=-1, half=-1):

    ipArr = np.ones((len(cleanArray), 4))
    ipArr[:, :3] = cleanArray[:, 3:6]
    return ipArr




def quantileCutNewArray(xyzArray, cut):

    if cut == 0:
        return xyzArray

    # calculate cut length
    cut = int(len(xyzArray) * (cut / 100))

    # don't use average, some values are far too large, median is a better estimation
    comMed = np.median(xyzArray[:, 0:3], axis=0)

    # sort by distance and cut largest
    distVec = xyzArray[:, 0:3] - comMed
    distVecNorm = np.linalg.norm(distVec, axis=1)
    xyzArray = xyzArray[distVecNorm.argsort()]
    xyzArray = xyzArray[:-cut]

    return xyzArray

def quantileCut(cleanArray, cut):

    if cut == 0:
        return cleanArray

    # calculate cut length
    cut = int(len(cleanArray) * (cut / 100))

    # don't use average, some values are far too large, median is a better estimation
    comMed = np.median(cleanArray[:, 3:6], axis=0)

    # sort by distance and cut largest
    distVec = cleanArray[:, 3:6] - comMed
    distVecNorm = np.linalg.norm(distVec, axis=1)
    cleanArray = cleanArray[distVecNorm.argsort()]
    cleanArray = cleanArray[:-cut]

    return cleanArray


if __name__ == "__main__":
    print('oh hai!')

    import matplotlib.pyplot as plt

    # latexmu = r'\textmu{}'
    # latexsigma = r'\textsigma{}'
    # latexPercent = r'\%'
    # plt.rc('font', **{'family': 'serif', 'serif': ['Palatino'], 'size': 10})
    # plt.rc('text', usetex=True)
    # plt.rc('text.latex', preamble=r'\usepackage[euler]{textgreek}')
    # plt.rcParams["legend.loc"] = 'lower right'

    # read some 10 files
    # this is the same as in the actual align code
    # box = 'box-0.0'

    # path = '/media/CacheDrive2TB/Arbeit/BackupOldData/VirtualDir/boxIPdist-uncut'
    # path = '/mnt/himsterData/roklasen/LumiFit/plab_1.50GeV/dpm_elastic_theta_2.7-13.0mrad_recoil_corrected/ip_offset_XYZDXDYDZ_0.0_0.0_0.0_0.0_0.0_0.0/beam_grad_XYDXDY_0.0_0.0_0.0_0.0/no_geo_misalignment/100000/1-100_xy_m_cut_real/no_alignment_correction'
    path = '/mnt/himsterData/roklasen/LumiFit/plab_1.50GeV/dpm_elastic_theta_2.7-13.0mrad_recoil_corrected/ip_offset_XYZDXDYDZ_0.0_0.0_0.0_0.0_0.0_0.0/beam_grad_XYDXDY_0.0_0.0_0.0_0.0/no_geo_misalignment/100000/1-100_uncut/no_alignment_correction'


    # file1 = f'{path}/1.5/{box}/Lumi_TrksQA_1000000.root'
    # file2 = f'{path}/15.0/{box}/Lumi_TrksQA_1000000.root'
    file1 = f'{path}/Lumi_TrksQA_1000000.root'
    file2 = f'{path}/Lumi_TrksQA_1000000.root'


    array1 = getIPnew(file1)
    array2 = getIPnew(file2)

    cut = 5
   

    ip1 =  quantileCutNewArray(array1,cut )* 10 
    ip2 =  quantileCutNewArray(array2 ,cut)* 10 

    print(f'averages: x:{np.mean(ip1[...,0])}, y:{np.mean(ip1[...,1])}' )

    mus1 = np.average(ip1, axis=0)
    mus2 = np.average(ip2, axis=0)
    sigmas1 = np.std(ip1, axis=0)
    sigmas2 = np.std(ip2, axis=0)

    xmin1 = np.min(ip1[:,0])
    xmax1 = np.max(ip1[:,0])
    ymin1 = np.min(ip1[:,1])
    ymax1 = np.max(ip1[:,1])

    xmin2 = np.min(ip2[:,0])
    xmax2 = np.max(ip2[:,0])
    ymin2 = np.min(ip2[:,1])
    ymax2 = np.max(ip2[:,1])

    xmin = min(xmin1, xmin2)
    ymin = min(ymin1, ymin2)
    xmax = max(xmax1, xmax2)
    ymax = max(ymax1, ymax2)

    thisrange = [[xmin, xmax], [ymin, ymax]]
    

    #* combined plots
    fig, axis = plt.subplots(1, 2, sharex=True, sharey=True)

    axis[0].hist2d(ip1[:, 0], ip1[:, 1], bins=100, norm=LogNorm(), rasterized=True, range=thisrange)
    # axis[0].set_xlabel(f'$i_x$ [mm]\n{latexmu}x={mus1[0]:.2f} [mm], {latexsigma}x={sigmas1[0]:.2f} [mm]')
    # axis[0].set_ylabel(f'$i_y$ [mm]\n{latexmu}y={mus1[1]:.2f} [mm], {latexsigma}y={sigmas1[1]:.2f} [mm]')
    axis[0].set_aspect('equal')


    axis[1].hist2d(ip2[:, 0], ip2[:, 1], bins=100, norm=LogNorm(), rasterized=True, range=thisrange)
    # axis[1].set_xlabel(f'$i_x$ [mm]\n{latexmu}x={np.round(mus2[0], 1)} [mm], {latexsigma}x={np.round(sigmas2[0], 1)} [mm]')
    # axis[1].set_ylabel(f'$i_y$ [mm]\n{latexmu}y={np.round(mus2[1], 1)} [mm], {latexsigma}y={np.round(sigmas2[1], 1)} [mm]')
    axis[1].yaxis.set_label_position('right')
    axis[1].yaxis.set_ticks_position('none')
    axis[1].set_aspect('equal')

    axis[0].set_xlim(xmin, xmax)
    axis[0].set_ylim(ymin, ymax)



    loc = plticker.MultipleLocator(round((xmax-xmin)/3)) # this locator puts ticks at regular intervals
    axis[0].xaxis.set_major_locator(loc)
    axis[0].yaxis.set_major_locator(loc)
    axis[1].xaxis.set_major_locator(loc)
    axis[1].yaxis.set_major_locator(loc)

    fig.set_size_inches((15 / 2.54, 8 / 2.54))
    fig.subplots_adjust(wspace=0, hspace=0)
    box = 'nobox'
    
    fig.savefig(f'output/ipDistribution/{box}-cut{cut}-both.pdf', dpi=300, bbox_inches='tight', pad_inches = 0.05)
    # fig
    plt.close(fig)

oh hai!
averages: x:-0.11632383945301232, y:-0.020112026305487016


In [12]:
import numpy as np
import sklearn.mixture

def fit(array):
    gmm = sklearn.mixture.GaussianMixture()
    r = gmm.fit(array[:, np.newaxis]) # GMM requires 2D data as of sklearn version 0.16
    print(f"mean : {r.means_}, var : {r.covariances_}" )

In [25]:
print('Mean and Gaussfit x Component:')
print(np.average(array1[:,0]))
fit(array1[:,0])
print('---------------------')
print('Mean and Gaussfit y Component:')
print(np.average(array1[:,1]))
fit(array2[:,1])

Mean and Gaussfit x Component:
-0.28945388288348983
mean : [[-0.289]], var : [[[379.36]]]
---------------------
Mean and Gaussfit y Component:
0.08167650564789421
mean : [[0.082]], var : [[[364.838]]]
